# Custom generative ai evaluator using python function wrapper on OpenAI model.

The notebook demonstrates the creation of custom generative ai evaluator by creating a python function deployment in Cloud/CPD which wraps an OpenAI model.
Here the OpenAI model is used as the evaluator. Any other model from providers like AWS or VertexAI etc.. can be used instead of the OpenAI model.

The custom generative ai evaluator endpoint should support the input, output formats described below.

Input format
```json
{
  "input_data": [
    {
      "fields": ["input"],
      "values": [["<prompt_1>"], ["<prompt_2>"]]
    }
  ]
}
```
e.g: `{"input_data": [{"fields": ["input"], "values": [["tell me about IBM"], ["tell me about openscale"]]}]}`

Output format
```json
{
  "predictions": [
    {
      "fields": ["generated_text"],
      "values": [
        [
          "<generated_text_value_1>"
        ],
        [
          "<generated_text_value_2>"
        ]
      ]
    }
  ]
```
e.g: `{"predictions": [{"fields": ["generated_text"], "values": [["International Business Machines Corporation (IBM) is a multinational technology company..."], ["IBM Watson OpenScale is a machine learning model ...."]]}]}`

**Note**: In the output response, generated_text field name is mandatory.

## Learning goals
- Configure OpenAI model 
- Create python function
- Deploy python function
- Test the Deployment

## Contents

- [Step 1 - Setup](#step-1)
- [Step 2 - Python function creation and deployment in watsonx.ai](#step-2)
- [Step 3 - Testing python function deployment](#step-3)

## Step 1 - Setup <a id="step-1"></a>

### Install the necessary libraries

In [ ]:
!pip install --upgrade ibm-watsonx-ai | tail -n 1
!pip install openai | tail -n 1

### Configure credentials

In [ ]:
CLOUD_API_KEY = "<API_KEY>"

CREDENTIALS = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY,
}

Uncomment the code and execute the cell below only if the Python function needs to be deployed in the CPD.

In [ ]:
# CREDENTIALS = {
#     "url": "<CPD_URL>",
#     "username": "<USERNAME>",
#     "password": "<PASSWORD>",
#     "instance_id": "openshift",
#     "apikey": "<API_KEY>",
#     "version": "5.0",
# }

In [2]:
from ibm_watsonx_ai import APIClient

watsonx_ai_client = APIClient(CREDENTIALS)
watsonx_ai_client.version

'1.1.22'

In [ ]:
space_id = "<DEPLOYMENT_SPACE_ID>"
watsonx_ai_client.set.default_space(space_id)

'SUCCESS'

### OpenAI model credentials

In [ ]:
OPENAI_CREDENTIALS = {
    "api_key": "<API_KEY>",
    "api_version": "<API_VERSION>",
    "azure_endpoint": "<ENDPOINT>",
}

## Step 2 - Python function creation and deployment in watsonx.ai <a id="step-2"></a>

This wrapper function asynchronously scores against the OpenAI `gpt-4` model. The model's response is then converted into the required output format.

In [5]:
def scoring_wrapper(params=OPENAI_CREDENTIALS):
    import subprocess
    import sys
    import asyncio

    try:
        import openai
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])
    finally:
        import openai

    chunk_size = 32

    def score(payload):
        client = openai.AsyncAzureOpenAI(
            api_key=params["api_key"],
            api_version=params["api_version"],
            azure_endpoint=params["azure_endpoint"],
        )
        values = payload["input_data"][0]["values"]
        inputs = [value[0] for value in values]

        async def score_async_wrap():
            async def send_completion(client, chunk):
                chunks = "\n".join(
                    [
                        f"{i}. {content}"
                        for i, content in enumerate(
                            inputs[chunk : chunk + chunk_size], chunk + 1
                        )
                    ]
                )
                chat_completion = await client.chat.completions.create(
                    model="gpt-4",
                    messages=[
                        {
                            "role": "system",
                            "content": "You will be given multiple numbered requests. For each one, provide an accurate and numbered response. Ensure that the number of responses exactly matches the number of requests, and format each response on a separate line corresponding to its request number.",
                        },
                        {"role": "user", "content": chunks},
                    ],
                )
                results = []
                responses = chat_completion.choices[0].message.content.split("\n")
                for response in responses:
                    star_rating = response.split(".", 1)[1].strip()
                    results.append(star_rating)
                return results

            tasks = [
                send_completion(client, chunk)
                for chunk in range(0, len(inputs), chunk_size)
            ]
            responses = await asyncio.gather(*tasks)
            results = []
            for response in responses:
                results += response
            return {
                "predictions": [
                    {
                        "fields": ["generated_text"],
                        "values": [[result] for result in results],
                    }
                ]
            }

        return asyncio.run(score_async_wrap())

    return score

### Storing python function

In [ ]:
sofware_spec_uid = watsonx_ai_client.software_specifications.get_id_by_name(
    "runtime-24.1-py3.11"
)

func_name = "<FUNCTION_NAME>"
meta_data = {
    watsonx_ai_client.repository.FunctionMetaNames.NAME: func_name,
    watsonx_ai_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
}

function_details = watsonx_ai_client.repository.store_function(
    meta_props=meta_data, function=scoring_wrapper
)

In [7]:
function_details

{'entity': {'software_spec': {'id': '45f12dfe-aa78-5b8d-9f38-0ee223c47309',
   'name': 'runtime-24.1-py3.11'},
  'type': 'python'},
 'metadata': {'created_at': '2024-11-08T11:01:01.975Z',
  'id': '2e942158-db7e-4fd2-bb5b-b1b583d194ae',
  'modified_at': '2024-11-08T11:01:01.975Z',
  'name': 'test_openai',
  'owner': 'IBMid-693000DYYL',
  'space_id': '74557a01-62df-49f8-9be1-571f7d26ee28'},
 'system': {'warnings': []}}

In [8]:
function_uid = function_details["metadata"]["id"]
print("Function UID:" + function_uid)

Function UID:2e942158-db7e-4fd2-bb5b-b1b583d194ae


### Deploying the function

In [9]:
function_deployment_details = watsonx_ai_client.deployments.create(
    function_uid,
    {
        watsonx_ai_client.deployments.ConfigurationMetaNames.NAME: func_name + "_deployment",
        watsonx_ai_client.deployments.ConfigurationMetaNames.ONLINE: {},
    },
)



######################################################################################

Synchronous deployment creation for id: '2e942158-db7e-4fd2-bb5b-b1b583d194ae' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='f77c54e4-a14c-4543-a269-170260a9343f'
-----------------------------------------------------------------------------------------------




In [10]:
func_deployment_uid = watsonx_ai_client.deployments.get_uid(function_deployment_details)
print("Function Deployment UID:" + func_deployment_uid)

Function Deployment UID:f77c54e4-a14c-4543-a269-170260a9343f


## Step 3 - Testing python function deployment <a id="step-3"></a>

In [11]:
func_scoring_url = watsonx_ai_client.deployments.get_scoring_href(function_deployment_details)
print("Scoring URL:" + func_scoring_url)

Scoring URL:https://us-south.ml.cloud.ibm.com/ml/v4/deployments/f77c54e4-a14c-4543-a269-170260a9343f/predictions


In [12]:
payload_scoring = {
    "input_data": [{"fields": ["input"], "values": [["hi"], ["what is 1+1"]]}]
}

scores_function_response = watsonx_ai_client.deployments.score(
    func_deployment_uid, payload_scoring
)
print(scores_function_response)

{'predictions': [{'fields': ['generated_text'], 'values': [['Hello!'], ['1+1 equals 2.']]}]}
